Neo4j Aura Connection Test

In [25]:
from neo4j import GraphDatabase

In [26]:
# Set up the Neo4j driver
uri = "neo4j+s://2c9df469.databases.neo4j.io"
user = "neo4j"
password = "qcykrYY8F9afv2Nz1kOKpK8Q0dSNAfL3RJBY1ejvIGY"
driver = GraphDatabase.driver(uri, auth=(user, password))


In [27]:
# Test the connection by running a simple query
with driver.session() as session:
    result = session.run("RETURN 1")
    print(result.single()[0])

# Close the driver connection
driver.close()

1


Upload data - test

In [51]:
# libraries
from py2neo import Graph, Node, Relationship

# Set up the Neo4j driver
graph = Graph(uri, auth=(user, password))

In [60]:
# Set up the Neo4j driver
graph = Graph(uri, auth=(user, password))

# Create some nodes and relationships
adam = Node("Person", name="Adam", age=70, last_name="Smith")
eve = Node("Person", name="Eve", age=68, last_name="Smith")
joe = Node("Person", name="Joe", age=42, last_name="Smith")
jane = Node("Person", name="Jane", age=40, last_name="Smith")
john = Node("Person", name="John", age=12, last_name="Smith")
jill = Node("Person", name="Jill", age=10, last_name="Smith")

father_of = Relationship.type("FATHER_OF")
mother_of = Relationship.type("MOTHER_OF")
spouse = Relationship.type("SPOUSE")

father = father_of(joe)
mother = mother_of(joe)
parents = spouse(adam, eve)
son = father_of(john)
daughter = mother_of(jill)

# Load the family tree into Neo4j
graph.begin().create(adam | eve | joe | jane | john | jill)
graph.begin().create(parents | father | mother | son | daughter)


In [61]:
#delete all nodes and relationships
graph.run("MATCH (n) DETACH DELETE n")

(No data)

In [29]:
# Define the family members
grandpa = Node("Person", name="Grandpa")
dad = Node("Person", name="Dad")
uncle = Node("Person", name="Uncle")
mom = Node("Person", name="Mom")
me = Node("Person", name="Me")
brother = Node("Person", name="Brother")
son = Node("Person", name="Son")
daughter = Node("Person", name="Daughter")

# Define the relationships
dad_rel = Relationship(grandpa, "FATHER_OF", dad)
uncle_rel = Relationship(grandpa, "FATHER_OF", uncle)
me_rel = Relationship(dad, "FATHER_OF", me)
brother_rel = Relationship(dad, "FATHER_OF", brother)
son_rel = Relationship(me, "FATHER_OF", son)
daughter_rel = Relationship(me, "MOTHER_OF", daughter)
married_rel = Relationship(dad, "MARRIED_TO", mom)

# Create the family tree
graph.create(grandpa)
graph.create(dad)
graph.create(uncle)
graph.create(mom)
graph.create(me)
graph.create(brother)
graph.create(son)
graph.create(daughter)
graph.create(dad_rel)
graph.create(uncle_rel)
graph.create(me_rel)
graph.create(brother_rel)
graph.create(son_rel)
graph.create(daughter_rel)
graph.create(married_rel)

JSON Upload test

In [10]:


# Open the JSON file

json_path = "C:\Users\Roberto\OneDrive\cargoIabono\Proyectos y Desarrollos\P001_V001_CFDI-Reader\03_Outputs\json_converter_test\0F23FE0D-8324-4BCE-AFAC-68CB67E89714.json"


with open(json_path) as f:
    data = json.load(f)

ValueError: embedded null character

In [ ]:

# Open the JSON file
with open(json_path) as f:
    data = json.load(f)

# Connect to the Neo4j database
driver = GraphDatabase.driver(uri, auth=(user, password))

# Define a function to create a node and its relationships
def create_node(tx, node):
    # Create the node
    tx.run("MERGE (n:{label} {{id: {id}}})",
           label=node['label'], id=node['id'])

    # Create relationships to its parent nodes
    for parent in node.get('parents', []):
        tx.run("MATCH (n:{label} {{id: {id}}}), (p:{parent_label} {{id: {parent_id}}}) "
               "MERGE (n)-[:CHILD_OF]->(p)",
               label=node['label'], id=node['id'],
               parent_label=parent['label'], parent_id=parent['id'])

# Process the JSON data and create the nodes and relationships in the Neo4j database
with driver.session() as session:
    for node in data:
        session.write_transaction(create_node, node)

# Close the driver connection
driver.close()
